In [2]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import selenium
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import csv



date_list = pd.Series(pd.date_range(datetime(2018,9,1), datetime.today()))
print(date_list)

0    2018-09-01
1    2018-09-02
2    2018-09-03
3    2018-09-04
4    2018-09-05
5    2018-09-06
6    2018-09-07
7    2018-09-08
8    2018-09-09
9    2018-09-10
10   2018-09-11
11   2018-09-12
12   2018-09-13
13   2018-09-14
14   2018-09-15
15   2018-09-16
16   2018-09-17
17   2018-09-18
18   2018-09-19
19   2018-09-20
20   2018-09-21
21   2018-09-22
22   2018-09-23
23   2018-09-24
24   2018-09-25
25   2018-09-26
26   2018-09-27
27   2018-09-28
28   2018-09-29
29   2018-09-30
30   2018-10-01
31   2018-10-02
32   2018-10-03
33   2018-10-04
34   2018-10-05
35   2018-10-06
36   2018-10-07
37   2018-10-08
38   2018-10-09
dtype: datetime64[ns]


In [8]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import selenium
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import csv


def xpath(row=0,col=0,sub=0):
    xpath = '//app/city-history/city-history-layout/div/div[2]/'\
            + 'section/div[2]/div[3]/div/div/div/div/'\
            + 'city-history-observation/div/div[2]/table'\
            + '/tbody/tr/td[' + str(col+1) + ']'\
            + '/table/tbody/tr[' + str(row+1) + ']'\
            + '/td[' + str(sub+1) + ']'
    return xpath


chromedriver = '/Users/grazillionaire/Documents/chromedriver' # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)


# Generate list of dates (months) to pull weather data for.

# For Chicago
#city = 'il/chicago/KORD'
#start_date = datetime(2001,1,1)
#end_date = datetime(2018,6,30)

# For New York
city = 'ny/new-york/KNYC'
start_date = datetime(2010,5,5)
end_date = datetime.today()

date_list = pd.Series(pd.date_range(start_date,end_date))
date_list = [datetime.today()] ################# delete this ##################
#city = 'ca/san-francisco/KSFO' ################ delete this ##################




for date in date_list:
    
    # Record start time (for scraper performance purposes)
    start_time = datetime.now()
    
    url = 'https://www.wunderground.com/history/monthly/us/{loc}/date/{y}-{m}?cm_ven=localwx_history'\
          .format(loc=city, y=date.year, m=date.month)
    driver.get(url)
    
    time.sleep(3)
    
    webpage_data = []

    # header stuff
    pass
    pass

    # rows
    row_idx = 3
    column_idx = 0
    subcolumn_idx = 0

    # Iterate through rows
    while True:
        # For each row,
        try:
            # ...try to read the subcolumn at the following xpath to see if the row exists.
            raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

            # If that worked, then initialize column_idx to 0 and row_data to [].
            column_idx = 0
            row_data = []

            # Iterate through columns
            while True:
                # For each column,
                try:
                    # ...try to read the subcolumn at the following xpath to see if the column exists.
                    raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

                    # If that worked, then initialize subcolumn_idx to 0 and column_data to [].
                    subcolumn_idx = 0
                    column_data = []

                    # Iterate through subcolumns
                    while True:
                        # For each subcolumn,
                        try:
                            # ...try to read the subcolumn at the following xpath to see if it exists.
                            raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

                            # If that worked, then try to convert raw_datum.text to a float.
                            try:
                                if column_idx == 0:
                                    datum = int(raw_datum.text) # Store dates as integers.
                                else:
                                    datum = float(raw_datum.text) # Store measured values as floats.
                            except:
                                datum = float('nan') # Store dashes and anything that doesn't cooperate as NaNs.
                            column_data.append(datum)

                        except:
                            # If it didn't work, there is no subcolumn at that index. Break subcolumn loop.
                            break

                        # next subcolumn
                        subcolumn_idx += 1

                    # That's it for the subcolumns in this column. Store column_data in row_data.
                    row_data.extend(column_data)

                except:
                    #if it didn't work, there is no column at that index. 
                    break

                # next column
                column_idx += 1

            # That's it for the columns in this row.

        except:
            # If it didn't work, there is no row at that index.
            break

        # next row
        row_idx += 1

    # That's it for the rows in the table. Store row_data in webpage_data. Done with this webpage!
    webpage_data.append(row_data)



    stop_time = datetime.now()
    time_diff = stop_time-start_time
    print('elapsed time:', time_diff.seconds + time_diff.microseconds/1000000, 's')
    print(webpage_data)


    myFile = open('example-NYC-{y}-{m}.csv'.format(y=date.year, m=date.month), 'w')
    with myFile:
        writer = csv.writer(myFile)
        writer.writerows(webpage_data)
    
driver.close()

elapsed time: 19.442603 s
[[3, 74.0, 69.0, 63.0]]


In [7]:
webpage_data

[[3, 74.0, 69.0, 63.0]]

In [5]:
12*18 + 12*8
312*160

49920

In [6]:
50000/3600

13.88888888888889